# **온라인 강의 플랫폼 구축하기**
**과제4번** <br>

1708164 김종호

# 과제4


---

요구사항: LR 클래스 내 원형 큐(Circular queue) 기반 Top k의 유사한 강좌를 추천하는 함수를 구현
하시오. 또한, 과제 #2와 #3, #4에서 구현한 함수들의 실행시간을 각각 출력하시오.
- 기존 함수 recommend_lectures(lec, k)는 코사인 유사도 값에 대해 내림차 정렬을 진행
한 뒤 TOP k개를 선택한다. 이 함수의 문제점은 비교할 강좌 수에 따라 계산시간이 비례하여
상승하는 문제이다. 이를 해결하기 위해, 우리는 최대 적재용량이 k인 원형 큐를 사용하여
TOP k의 강좌를 추출하는 recommend_lectures_using_circular_queue(lec, k)을
구현한다.

◼ 함수 recommend_lectures_using_circular_queue(lec, k) 은 recommend_lectures 함
수와 유사하지만, 코사인 유사도의 특정 값 이상 만족하는 k개의 강좌만 추출한다. 구체적
으로, 코사인 유사도 0.005이상을 만족하는 강좌는 후보군에 넣고, 그 이하의 값은 버린
다. 최대 적재용량에 도달한 경우, 남은 강좌를 추가적으로 비교하지 않고 원형 큐에 저장
된 k개만 추천한다. 여기서, 모든 강좌를 비교했음에도 불구하고도 m개 강좌(0≤m<k) 만
큐에 저장된 경우, m개 강좌를 주어진 강좌의 유사한 강좌 후보로 추천한다.
- 과제 #2, #3, #4에서 생성된 함수들의 개별적인 실행시간(execution time)을 출력하도록 세부
사항을 참고하여 함수를 개선하시오.

◼ 데코레이터 패턴은 장식자 패턴이라고도 하며, 새로운 클래스 생성이나 객체의 행동에 영
향을 끼치는 영향들이 없이 클래스의 기능을 동적으로 추가하는 방법이다. 그림 2는 데코
레이터 패턴(decorator pattern)의 예제 코드이다. 해당 코드는 두 번 함수를 호출하는
@my_wrapper 의 데코레이터를 사용하여 my_hello 함수를 두 번 호출하도록 확장한다.

◼ 통상적으로, 함수의 실행시간은 함수를 실행하기 시작 직전시간과 종료 직후시간 간의 차
이인 경과된 시간(Elapsed time)으로 계산된다. 그림 3 내 코드를 참고하시오.

◼ 실행시간이 필수적으로 출력되어야 하는 함수 리스트는 recommend_lectures,
recommend_lectures_using_circular_queue, loads, search, delete 이다. 즉, 학생들은 그림 2과 3의 코드를 응용하여 주어진 리스트의 각 함수의 실행전에
@timer 의 표기(annotation)만 함으로써 실행시간이 출력되도록 하시오.

In [307]:
#데코레이터 패턴, 실행시간 체크
import time
def timer(fn):
  def inner(*args, **kwargs):
    start_time=time.time()
    fn(*args, **kwargs)
    elapsed=time.time()-start_time   
    print("execution time: {}".format(elapsed*1000)+"\n")
    return fn(*args, **kwargs) #데코레이터로 꾸며준 함수 리턴
  return inner

In [308]:
from typing_extensions import ParamSpecArgs
from array import array
import json
import math
import re
from collections import Counter


# 정규표현식을 위한 WORD
WORD = re.compile(r"\w+")

#전체출력
def print_all(head,data_container):
 if(data_container=='SLL'): #Singly Linked List
    curNode=head
    count=0
    while curNode is not None: 

      print(curNode.name)
      curNode=curNode.next
      count=count+1
 elif(data_container=='CLL'): #Circular Linked List
  #  count=0
   curNode=head
   done = curNode is None
   while not done:
    #  count=count+1
     print(curNode.name)
     curNode=curNode.next 
     done=curNode is head
  #  print("count?",count)


def text_to_vector(text):
  '''
  텍스트를 단어(word)로 나눈뒤, 각 단어별로 카운터하여 Dict 형태로 저장
  @text : 텍스트
  '''
  words = WORD.findall(text)
  return Counter(words)

class OCP:
  '''
   Online Course Platform 클래스
  '''
  # ocp는 IMDF, LDM, LR, PM 의 클래스의 인스턴스 포함.

  def __init__(self):
    self.LR=LR()
    self.IMDF=IMDF()
    self.LDM=LDM()
    self.PM=PM()
    pass
  

  def loads(self,filename,data_container): # 추가
    LDM.store_lectures_from_json(self,filename,data_container,self.IMDF)
    
  def search(self,target): #검색
    LDM.search(self,target,self.IMDF)


  def delete(self, target): #삭제
    LDM.delete(self,target,self.IMDF)

class Lecture:
  '''
   OCP 플랫폼 내부의 정보교환을 위한 기본 데이터 모델
  '''
  #생성자
  def __init__(self, blocks_url,id,name,data):
    self.blocks_url=blocks_url # 강좌 URL
    self.id=id #강좌 아이디
    self.name=name #강좌명
    self.data=data # 각 강좌별 데이터
    self.next=None # 다음 노드에 대한 link

  #강좌 아이디, 강좌명과 강좌 URL 출력
  def __str__(self):
    return f" 강좌 URL: { self.blocks_url}, 강좌아이디 : {self.id}, 강좌명 : {self.name} "


class IMDF:
  '''
  인메모리 데이터 패프릭
  강의 데이터를 관리하는 다양한 종류의 데이터 구조를 제공하는 컴포넌트
  '''
  # head_reference=None
  def __init__(self):
   self.head_reference=None
   pass


class LDM:
  head=None
  '''
  대응되는 데이터 구조에 따라 강의데이터를 검색,정렬,추가,삭제가 가능한 컴포넌트
  '''
  def __init__(self):
    pass
  @timer
  def store_lectures_from_json(self,filename,data_container,IMDF): #추가
    with open(filename,'r',encoding="UTF-8") as f:
     self.result= json.load(f) #200개 data
    
    #json 파일을 읽으면, 딕셔너리 형태로 변수에 저장된다.

    # print(self.IMDF.head_reference.id)
    if(data_container=='SLL'):
      IMDF.head_reference=Lecture(self.result[199]['blocks_url'],self.result[199]['id'],self.result[199]['name'],self.result[199])
      for i in range(198,-1,-1):
        newNode=Lecture(self.result[i]['blocks_url'],self.result[i]['id'],self.result[i]['name'],self.result[i])
        newNode.next=IMDF.head_reference
        IMDF.head_reference=newNode  
     #print(self.result[0]['id'])
     
    elif(data_container=='CLL'): #circular linked list
        IMDF.head_reference=Lecture(self.result[0]['blocks_url'],self.result[0]['id'],self.result[0]['name'],self.result[0])
        copy_head_reference=IMDF.head_reference
        IMDF.head_reference.next=IMDF.head_reference   
        for i in range(1,200):
          newNode=Lecture(self.result[i]['blocks_url'],self.result[i]['id'],self.result[i]['name'],self.result[i])
          # print(newNode.name)
          IMDF.head_reference.next=newNode
          newNode.next=copy_head_reference
          IMDF.head_reference=newNode
        # print(IMDF.head_reference.next.name)
          
  @timer
  def search(self,name,IMDF): #검색
    curNode=IMDF.head_reference

    while curNode is not None and curNode.name!=name:
      curNode=curNode.next
    
    if curNode is None:
      print(name+ "이란 강좌명은 없습니다.")
      return
    else:
      return print("강좌명: "+curNode.name +", 강좌URL: "+ curNode.blocks_url+ ", 강좌ID: "+curNode.id)
    
  @timer
  def delete(self,name,IMDF):
    preNode=None
    curNode=IMDF.head_reference 

    while curNode is not None and curNode.name!=name:
      preNode=curNode
      curNode=curNode.next

    if curNode is not None:
      if curNode is IMDF.head_reference:
        IMDF.head_reference=curNode.next
      else:
        preNode.next=curNode.next
  
        
# sl=LDM()
# sl.store_lectures_from_json('/content/sample_data/lectures.json')

class LR:
 '''
 강의 데이터 조작 모듈과 함께 적절한 강의 데이터를 추천하는 컴포넌트
 '''
 def get_cosine_simiarity(self,lecture1, lecture2): 
    '''
    코사인 값을 가져오는 함수
    @vec1: 벡터 1
    @vec2: 벡터 2
    '''
    lecture1=text_to_vector(lecture1)
    lecture2=text_to_vector(lecture2)

    intersection = set(lecture1.keys()) & set(lecture2.keys())
    numerator = sum([lecture1[x] * lecture2[x] for x in intersection])
    sum1 = sum([lecture1[x] ** 2 for x in list(lecture1.keys())])
    sum2 = sum([lecture2[x] ** 2 for x in list(lecture2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
      return 0.0
    else:
      return float(numerator) / denominator 
  
 def __init__(self):
   pass
 @timer
 def recommend_lectures(self,lec, k):  #주어진 강좌와 유사한 강의 TOP k개를 추천하는 함수 
   lecture_dict={}
   curNode=lec
   done = curNode is None
   while not done:
      curNode=curNode.next
      lecture2=curNode.data['short_description']
      consine=self.get_cosine_simiarity(lec.data['short_description'],lecture2)
      if consine==1.0: # 동일 강좌 배제
        done=curNode is lec
        continue
      else:
        lecture_dict[consine]=curNode.name
        done=curNode is lec

   sdict = sorted(lecture_dict.items(),reverse=True)
   count=0
   answer=[]
   for key ,value in sdict:
    #  print(key) 유사도확인
     if count==k:
       break
     answer.append(value)
     count=count+1
   return answer
  #  print(lecture_dict.values())
  #  for i in lecture_dict:
  #    print(i)
    
 
 @timer
 def recommend_lectures_using_circular_queue(self,lec,k): #과제4번 원형 큐기반 Top k의 유사한 강좌를 추천하는 함수
    # 최대용량 k인 원형큐 사용
    # 코사인 유사도 0.005이상 만족하는 값만 후보군 .
    # 최대 적재용량 도달하면 추가로 비교하지말고 바로 return
    # if 모든 강좌 비교했는데도 k개가 안되면 그냥 리턴

    Cque=Circular_queue(k)
    curNode=lec
    done= curNode is None
    result=list()
    # count=0
    while not done:
      # count+=1 #몇 번 도는지
      curNode=curNode.next
      lec2=curNode.data['short_description']
      consine=self.get_cosine_simiarity(lec.data['short_description'],lec2)
      if consine>=0.005 and consine!=1.0:
          Cque.enqueue(curNode.data['name'])
        
          if Cque.isFull():
            # print(count) # 몇 번만에 찾았는지 확인
            for i in range(0,k):
              # print(Cque.dequeue())
              result.append(Cque.dequeue())
            return result
        
      done=curNode is lec
   
    if(Cque.isEmpty()):
      print("조건에 맞는 유사한 강좌가 존재하지 않습니다.")
    while (not Cque.isEmpty()):
      result.append(Cque.dequeue())
    return result
    
class PM:
  '''
  OCP 플랫폼에서 동작하는 컴포넌트의 성능을 수행시간 측면에서 모니터링하고 가장 성능이 좋은 알고리즘을 확인
  '''
  def Monitoring_component(self,program):
    '''
    @param program : 성능을 측정할 프로그램
    '''
  def __init__(self):
   pass  


In [309]:
#원형큐
class Circular_queue:
  def __init__(self,maxSize):
    self._count=0
    self._front=0
    self._back=maxSize-1
    self._qArray=[None]*maxSize

  def isEmpty(self):
    return self._count==0


  def isFull(self):
    return self._count==len(self._qArray)

  def __len__(self):
    return self._count

  #add
  def enqueue(self,item):
    if self.isFull():
      raise IndexError('Queue full')
    maxSize=len(self._qArray)
    self._back=(self._back+1)%maxSize
    self._qArray[self._back]=item
    self._count+=1

  #remove
  def dequeue(self):
    if self.isEmpty():
      raise IndexError('Queue empty')
    item=self._qArray[self._front]
    maxSize=len(self._qArray)
    self._front=(self._front+1)%maxSize
    self._count-=1
    return item
  

In [311]:
OCP1=OCP() #OCP는 LDM과 함께 데이터 추가 및 검색 삭제의 예제로써 loads, search, delete 함수를 갖는다.
OCP1.loads('/content/sample_data/lectures.json','CLL')
# cosine = OCP1.LR.get_cosine_simiarity(OCP1.IMDF.head_reference.next.data['short_description'],OCP1.IMDF.head_reference.next.next.data['short_description'])
# # print("Cosine Similarity:", cosine)

# recommend_lecture=OCP1.LR.recommend_lectures(OCP1.IMDF.head_reference.next.next,10)
# print(recommend_lecture)
# print(OCP1.IMDF.head_reference.data['short_description'])
# print(OCP1.IMDF.head_reference.next.next.data['short_description'])
# print_all(OCP1.IMDF.head_reference,'CLL')



#원형 큐를 이용해서 Top k의 유사한 강좌를 추천하는 함수
OCP1.LR.recommend_lectures_using_circular_queue(OCP1.IMDF.head_reference.next.next.next,7)

execution time: 14.385461807250977

execution time: 1.7299652099609375



['민족의 지성 다산 정약용의 삶과 철학',
 '민족의 지성 다산 정약용의 삶과 철학',
 '전기자기학',
 '5분 안에 톺아보는 ‘북한의 민족유산(문화재)’',
 '마음챙김으로 스트레스 다스리기',
 '마음챙김으로 스트레스 다스리기',
 '우리 아이를 위한 아동안전설명서']

In [312]:
OCP2=OCP() 
OCP2.loads('/content/sample_data/lectures.json','SLL')

OCP2.search('우리말 형태론') 
OCP2.delete('우리말 형태론')



execution time: 8.561849594116211

강좌명: 우리말 형태론, 강좌URL: http://www.kmooc.kr/api/courses/v1/blocks/?course_id=course-v1%3AANU%2Banu_01%2B2019_T1_2, 강좌ID: course-v1:ANU+anu_01+2019_T1_2
execution time: 0.09083747863769531

강좌명: 우리말 형태론, 강좌URL: http://www.kmooc.kr/api/courses/v1/blocks/?course_id=course-v1%3AANU%2Banu_01%2B2019_T1_2, 강좌ID: course-v1:ANU+anu_01+2019_T1_2
execution time: 0.003814697265625

